In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import corner
import batman
import emcee
import time
from multiprocessing import Pool
from IPython.display import display, Math
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as tck

from exotic_ld import StellarLimbDarkening

import pandas as pd
import pickle

import celerite2
from celerite2 import terms


In [ ]:
TIC = 'TIC 66818296' #WASP-17 / TIC 66818296

search_result = lk.search_lightcurve(
    target=TIC,
    mission='TESS', author='SPOC')
print("trying to download")
lc_collection = search_result.download_all()
lc_sector_12 = lc_collection[0].remove_nans()
lc_sector_38 = lc_collection[1].remove_nans()

print("Search done")
print(lc_collection)

lc = lc_collection.stitch(corrector_func=lambda x: x.
                          remove_nans().normalize(unit='unscaled'))
#Sector 12 and sector 38
lc = lc.remove_nans()

In [ ]:
lc

In [ ]:
t = np.array(lc.time.btjd)
flux_data = np.array(lc.flux)
yerrs = np.array(lc.flux_err)

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,6))
plt.errorbar(t, flux_data, yerrs, fmt='.', color='Black', ecolor='LightGrey')


plt.ylabel("Normalised flux", size=17)
plt.xlabel("Time - 2457000 (BTJD days)", size=17)

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()


In [ ]:
def set_params(params, yerrs):
    _sigma, _rho = params
    kernel = terms.Matern32Term(sigma = _sigma, rho  = _rho)
    gp = celerite2.GaussianProcess(kernel, mean=0.0,)
    gp.compute(t, yerr=yerrs, quiet=True)

    return gp

In [ ]:
filename = "matern32_tess_flat_samples.p"
with open(filename, "rb") as f:
    flat_samples = pickle.load(f)
f.close()

In [ ]:
model_sigma = np.percentile(flat_samples[:, 0], 50)
model_rho = np.percentile(flat_samples[:, 1], 50)

In [9]:
model_parmas = [model_sigma, model_rho]
gp = set_params(model_parmas, yerrs)
gp.compute(t, yerr=yerrs)

conditional = gp.condition(flux_data, t)

fig, ax = plt.subplots(1, figsize=(8,6))

plt.plot(t, conditional.sample(), color="C1")
plt.errorbar(t, flux_data, yerrs, fmt='.', color='Black', ecolor='LightGrey')
plt.ylabel("Normalised flux", size=17)
plt.xlabel("Time - 2457000 (BTJD days)", size=17)

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6))

#Sector 12
ax1.set_title("Sector 12", size=17)
#Binned data?
ax1.errorbar(t, flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0) #Black
ax1.plot(t, conditional.sample(), color='Red', zorder=10)
ax1.set_xlim(min(lc_sector_12.time.btjd), max(lc_sector_12.time.btjd))

# Mirror y
ax1A = ax1.twinx()
ax1A.set_ylim(ax1.get_ylim())
ax1A.get_yaxis().set_ticklabels([])
ax1A.tick_params(which="both", direction="in")#, labelright=True, labelleft=True)

# Mirror x
ax1B = ax1.twiny()
ax1B.set_xlim(ax2.get_xlim())
ax1B.get_xaxis().set_ticklabels([])
ax1B.tick_params(which="both", direction="in")


#Sector 38
ax2.set_title("Sector 38", size=17)
#Binned data?
ax2.errorbar(t, flux_data, yerr=yerrs,fmt='.', markersize=5, color='black', ecolor='LightGrey', label="Data", zorder=0) #Black
ax2.plot(t, conditional.sample(), color='Red', zorder=10)
ax2.set_xlim(min(lc_sector_38.time.btjd), max(lc_sector_38.time.btjd))
ax2.set_ylim(ax1.get_ylim()) #Mirror ax1 ylims

ax2.get_yaxis().set_ticklabels([])
ax2.tick_params(axis="y",which="both", direction="in")

# Mirror y
ax2A = ax2.twinx()
ax2A.set_ylim(ax2.get_ylim())
ax2A.get_yaxis().set_ticklabels([])
ax2A.tick_params(which="both", direction="in")

# Mirror x
ax2B = ax2.twiny()
ax2B.set_xlim(ax2.get_xlim())
ax2B.get_xaxis().set_ticklabels([])
ax2B.tick_params(which="both", direction="in")



ax1.set_ylabel("Normalised flux", size=17)
ax1.set_xlabel("Time - 2457000 (BTJD days)", size=17)

ax1.yaxis.set_tick_params(labelsize=17)
ax1.xaxis.set_tick_params(labelsize=17)
ax2.xaxis.set_tick_params(labelsize=17)


ax1.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax1B.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax1.yaxis.set_minor_locator(tck.AutoMinorLocator())
ax1A.yaxis.set_minor_locator(tck.AutoMinorLocator())


ax2.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax2B.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax2.yaxis.set_minor_locator(tck.AutoMinorLocator())
ax2A.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.subplots_adjust(wspace=0)
plt.show()
